In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# Load classifier
import pickle
with open('/content/drive/MyDrive/fake_news_model/classifier_model.pkl', 'rb') as f:
    clf = pickle.load(f)

# Load tokenizer
from transformers import DistilBertTokenizer, DistilBertModel
tokenizer = DistilBertTokenizer.from_pretrained('/content/drive/MyDrive/fake_news_model/distilbert_tokenizer')

# Load model
model = DistilBertModel.from_pretrained('distilbert-base-uncased')
model.eval()



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Check contents of the folder
!ls /content/drive/MyDrive/fake_news_model/


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
classifier_model.pkl  distilbert_tokenizer


In [4]:
import pickle
import numpy as np
from transformers import DistilBertTokenizer, DistilBertModel
import torch


In [5]:
with open('/content/drive/MyDrive/fake_news_model/classifier_model.pkl', 'rb') as f:
    clf = pickle.load(f)


In [6]:
tokenizer = DistilBertTokenizer.from_pretrained('/content/drive/MyDrive/fake_news_model/distilbert_tokenizer')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')  # Or load a saved model path if you saved it
model.eval()

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [8]:
import pandas as pd

test_df = pd.read_csv('/content/testing_data_lowercase_nolabels.csv',sep='\t', header=None, names=['label', 'text'])  # adjust file name
texts = list(test_df['text'])  # replace with actual column name
test_df.head()


,label,text
0,2,copycat muslim terrorist arrested with assault...
1,2,wow! chicago protester caught on camera admits...
2,2,germany's fdp look to fill schaeuble's big shoes
3,2,mi school sends welcome back packet warning ki...
4,2,u.n. seeks 'massive' aid boost amid rohingya '...


In [9]:
batch_size = 16
test_embeddings = []

with torch.no_grad():
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        encoded = tokenizer(batch_texts, padding=True, truncation=True, max_length=128, return_tensors='pt')
        outputs = model(**encoded)
        cls_embeddings = outputs.last_hidden_state[:, 0, :]
        test_embeddings.extend(cls_embeddings.numpy())

X_test = np.array(test_embeddings)


In [10]:
predictions = clf.predict(X_test)
print(predictions)


[0 0 1 ... 1 0 0]


In [11]:
test_df['predicted_label'] = predictions

In [12]:
test_df

,label,text,predicted_label
0,2,copycat muslim terrorist arrested with assault...,0
1,2,wow! chicago protester caught on camera admits...,0
2,2,germany's fdp look to fill schaeuble's big shoes,1
3,2,mi school sends welcome back packet warning ki...,0
4,2,u.n. seeks 'massive' aid boost amid rohingya '...,1
...,...,...,...
9979,2,boom! fox news leftist chris wallace attempts ...,0
9980,2,here it is: list of democrat hypocrites who vo...,0
9981,2,new fires ravage rohingya villages in northwes...,1
9982,2,meals on wheels shuts the lyin‚ lefties up wit...,0


In [14]:
test_df.to_csv("test_predictions_Embedding.csv", index=False)

In [15]:
from google.colab import files
files.download("test_predictions_Embedding.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>